# FINAL PROJECT

In [ ]:
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from PIL import ImageTk,Image

import cv2
import numpy as np

Window = Tk()

Window.title("LEAF DISEASE DETECTION TEST")


Window.geometry("720x720")  # Set Window size

Window.configure(background='powderblue')  # Set background to powder blue

a_Label = Label(Window, text="LEAF DISEASE DETECTION",padx=100, bg="powderblue", fg="black"
                  , font=("arial", 25, "bold"))

mainLabel = Label(Window, text="YOGANANDA COLLEGE OF ENGINEERING AND TECHNOLOGY", padx=20, bg="powderblue", fg="black"
                  , font=("arial", 35, "bold","underline"))
mainLabel.pack()

a_Label.pack()

canvas=Canvas(Window,width=300,height=300)
canvas.pack()
img=ImageTk.PhotoImage(Image.open("test.png"))

canvas.create_image(20,20,anchor=NW,image=img)

Yellow_value = StringVar()
Brown_value = StringVar()
defected = StringVar()

def cal_MaxX_MaxY(width, height, mask):
    Max_x = 0
    Max_y = 0

    for i in range(0, height):  # Detect length of whites on X-axis
        Temp = 0
        for j in range(0, width):
            if mask[i, j] == 255:
                Temp += 1


        if Temp > Max_x:
            Max_x = Temp

    for i in range(0, width):  # Detect length of whites on X-axis
        Temp = 0
        for j in range(0, height):
            if mask[j, i] == 255:
                Temp += 1

        if Temp > Max_y:
            Max_y = Temp

    return Max_x, Max_y


def main(img_rec):

    IMG = img_rec
    IMG = cv2.resize(IMG, (400, 400))
    hsv = cv2.cvtColor(IMG, cv2.COLOR_BGR2HSV)

    lower_range = np.array([21, 8, 63])  # Works for BACTERIAL LEAF (100% Accuracy)
    upper_range = np.array([30, 255, 255])

    Ylower_range = np.array([17, 100, 100])  # For detecting yellow in paddy blast
    Yupper_range = np.array([23, 255, 255])

    Blower_range = np.array([10, 80, 40])  # Works for separating brown spots and bacterial leaf
    Bupper_range = np.array([20, 255, 255])

    maskLB = cv2.inRange(hsv, lower_range, upper_range)
    maskY = cv2.inRange(hsv, Ylower_range, Yupper_range)
    maskB = cv2.inRange(hsv, Blower_range, Bupper_range)
    tempY = maskY - maskB
    tempB = maskB - maskY

    heightLB, widthLB = maskLB.shape[:2]
    heightY, widthY = maskY.shape[:2]
    heightB, widthB = maskB.shape[:2]
    heightTY, widthTY = tempY.shape[:2]
    heightTB, widthTB = tempB.shape[:2]

    print("heightLB =", heightB)

    LBmax_x, LBmax_y = cal_MaxX_MaxY(widthLB, heightLB, maskLB)
    Ymax_x, Ymax_y = cal_MaxX_MaxY(widthY, heightY, maskY)
    Bmax_x, Bmax_y = cal_MaxX_MaxY(widthB, heightB, maskB)
    TYmax_x, TYmax_y = cal_MaxX_MaxY(widthTY, heightTY, tempY)
    TBmax_x, TBmax_y = cal_MaxX_MaxY(widthTB, heightTB, tempB)

    print("light brown LBmax_x =", LBmax_x)
    print("light brown LBmax_y =", LBmax_y)
    print("yellow to light brown Ymax_x =", Ymax_x)
    print("yellow to light brown Ymax_y =", Ymax_y)
    print("brown to black Bmax_x =", Bmax_x)
    print("brown to black Bmax_y =", Bmax_y)
    print("only yellow TYmax_x =", TYmax_x)
    print("only yellow TYmax_y =", TYmax_y)
    print("only brown TBmax_x =", TBmax_x)
    print("only brown TBmax_y =", TBmax_y)

    # print Ymax_x
    print("Yellow = ", TYmax_y)
    #print "Yellowx = ", TYmax_x
    print("Brown = ", TBmax_y)
    #print "Brownx  = ", TYmax_x

    Yellow_value.set(TYmax_y)
    Brown_value.set(TBmax_y)

    # leaf defected string value passing

    if TYmax_y >= 5 or TBmax_y >= 5:
        messagebox.showinfo("Result",'LEAF IS DEFECTED')
       
    else:
        messagebox.showinfo("Result",'LEAF IS HEALTHY')
    
    #if TYmax_y == 0 and TBmax_y == 0:
     #   defected.set("Normal Leaf")
    #if TYmax_y > 0 and TBmax_y == 0:
     #   defected.set("Defect started")
    #if TYmax_y >= 0 and TBmax_y > 0 and TBmax_y <= 26:
     #   defected.set("Medium defected")
    #if TYmax_y >= 0 and TBmax_y > 26:
     #   defected.set("Very much defected")

def image_disk():

    Tk().withdraw()  # we don't want a full GUI, so keep the root window from appearing
    filename = filedialog.askopenfilename()  # show an "Open" dialog box and return the path to the selected file
    img = cv2.imread(filename)
    main(img)

def image_camera():

    Cam = cv2.VideoCapture(0)

    cv2.namedWindow("test")

    Img_Counter = 0

    while True:
        ret, frame = Cam.read()
        cv2.imshow("test", frame)
        if not ret:
            break
        k = cv2.waitKey(1)

        if k % 256 == 27:  # PRESS ESCAPE TO CLOSE THE WEBCAM WINDOW
            # Enter pressed
            print("Escape hit, closing...")
            break

        elif k % 256 == 10:  # TAKE IMAGE PY PRESSING ENTER (TAKE ONLY 1 AT A TIME)
            # SPACE pressed
            img_name = str(Img_Counter) + ".png"
            cv2.imwrite(img_name, frame)
            print("{} written!".format(img_name))
            Img_Counter += 1

    Cam.release()

    cv2.destroyAllWindows()

    my_img = cv2.imread("0.png")  # CV2 will only read image named "0.png"

    main(my_img)

btn1 = Button(Window,
              text="Select Image From Disk",
              fg="black",
              command=image_disk, padx=10, pady=10, font=("arial", 20, "bold"), relief=SUNKEN, bd=4)

btn1.place(relx=0.5, rely=0.5, anchor=CENTER)

btn1.pack(side=TOP, pady=30)

btn2 = Button(Window,
              text="Take Image from WebCam",
              fg="black", padx=10, pady=10, font=("arial",20,"bold"),
              command=image_camera,
              relief=SUNKEN, bd=4)

btn2.place(relx=1, rely=1, anchor=CENTER)

btn2.pack(side=TOP, pady=10)

b_Label = Label(Window, text="ASIF AYOUB DAR", bg="powderblue", fg="black"
                  , font=("arial", 15, "bold"))

b_Label.place(relx=1, anchor=NE)
#b_Label.pack(side=BOTTOM, padx=5)


b_Label.pack(side=BOTTOM, padx=5)



#l1 = Label(root, text="Yellow color present")

#l2 = Label(root, textvariable=Yvalue,padx=10)

#l3 = Label(root, text="Brown color present")

#l4 = Label(root, textvariable=Bvalue, padx=10)

l5 = Label(Window, textvariable=defected, padx=10)

#l1.pack(pady=10)

#l2.pack()

#l3.pack(pady=10)

#l4.pack()

l5.pack(pady=10)

Window.mainloop()
